In [5]:
import os
import pandas as pd
import oandapyV20
import oandapyV20.endpoints.instruments as instruments
from datetime import datetime, timedelta
from oandapyV20.exceptions import V20Error

# -------------------
# Configuration
# -------------------
ACCESS_TOKEN = "b87068a621d73fd53e73f5044affc38f-5f46a213c43ee6ac9108d835ff0103d4"  # <== Paste your API key here
ACCOUNT_TYPE = "practice"  # "practice" or "live"
START_DATE = "2010-01-01"
END_DATE = "2024-12-31"
GRANULARITY = "D"  # Daily candles

# Create output folder
os.makedirs("data", exist_ok=True)

# 8 Currency Pairs (Majors + Crosses)
INSTRUMENTS = [
    "EUR_USD", "USD_JPY", "GBP_USD", "USD_CHF", "USD_CAD", "AUD_USD", "NZD_USD",
    "EUR_JPY", "GBP_JPY", "AUD_JPY", "NZD_JPY",
    "EUR_GBP", "EUR_AUD", "AUD_NZD", "CAD_JPY", "CHF_JPY",
    "EUR_CAD", "GBP_CAD", "EUR_CHF", "GBP_CHF"
]

# -------------------
# OANDA Client
# -------------------
client = oandapyV20.API(access_token=ACCESS_TOKEN, environment=ACCOUNT_TYPE)

# -------------------
# Fetch Daily Candles
# -------------------
def fetch_candles(symbol, start, end):
    all_data = []
    dt_start = pd.to_datetime(start)
    dt_end = pd.to_datetime(end)

    while dt_start < dt_end:
        dt_chunk_end = min(dt_start + timedelta(days=5000), dt_end)  # Max 5000 candles per request

        params = {
            "from": dt_start.strftime("%Y-%m-%dT%H:%M:%SZ"),
            "to": dt_chunk_end.strftime("%Y-%m-%dT%H:%M:%SZ"),
            "granularity": GRANULARITY,
            "price": "M"
        }

        try:
            r = instruments.InstrumentsCandles(instrument=symbol, params=params)
            client.request(r)
        except V20Error as e:
            print(f"Error fetching {symbol}: {e}")
            break

        candles = r.response.get("candles", [])

        for c in candles:
            if c["complete"]:
                all_data.append({
                    "time": c["time"],
                    "open": float(c["mid"]["o"]),
                    "high": float(c["mid"]["h"]),
                    "low": float(c["mid"]["l"]),
                    "close": float(c["mid"]["c"]),
                    "volume": c["volume"]
                })

        dt_start = dt_chunk_end

    df = pd.DataFrame(all_data)
    df["time"] = pd.to_datetime(df["time"])
    df.set_index("time", inplace=True)
    return df

# -------------------
# Run for All Symbols
# -------------------
for symbol in INSTRUMENTS:
    print(f"Fetching: {symbol}")
    df = fetch_candles(symbol, START_DATE, END_DATE)
    output_file = f"data/daily/{symbol}.csv"
    df.to_csv(output_file)
    print(f"Saved: {output_file}")


Fetching: EUR_USD
Saved: data/daily/EUR_USD.csv
Fetching: USD_JPY
Saved: data/daily/USD_JPY.csv
Fetching: GBP_USD
Saved: data/daily/GBP_USD.csv
Fetching: USD_CHF
Saved: data/daily/USD_CHF.csv
Fetching: USD_CAD
Saved: data/daily/USD_CAD.csv
Fetching: AUD_USD
Saved: data/daily/AUD_USD.csv
Fetching: NZD_USD
Saved: data/daily/NZD_USD.csv
Fetching: EUR_JPY
Saved: data/daily/EUR_JPY.csv
Fetching: GBP_JPY
Saved: data/daily/GBP_JPY.csv
Fetching: AUD_JPY
Saved: data/daily/AUD_JPY.csv
Fetching: NZD_JPY
Saved: data/daily/NZD_JPY.csv
Fetching: EUR_GBP
Saved: data/daily/EUR_GBP.csv
Fetching: EUR_AUD
Saved: data/daily/EUR_AUD.csv
Fetching: AUD_NZD
Saved: data/daily/AUD_NZD.csv
Fetching: CAD_JPY
Saved: data/daily/CAD_JPY.csv
Fetching: CHF_JPY
Saved: data/daily/CHF_JPY.csv
Fetching: EUR_CAD
Saved: data/daily/EUR_CAD.csv
Fetching: GBP_CAD
Saved: data/daily/GBP_CAD.csv
Fetching: EUR_CHF
Saved: data/daily/EUR_CHF.csv
Fetching: GBP_CHF
Saved: data/daily/GBP_CHF.csv


In [8]:
import pandas as pd
import os

DATA_FOLDER = "data/daily"

for filename in os.listdir(DATA_FOLDER):
    if filename.endswith(".csv"):
        filepath = os.path.join(DATA_FOLDER, filename)
        print(f"Cleaning: {filename}")
        
        # Load CSV and parse datetime
        df = pd.read_csv(filepath, parse_dates=["time"])
        
        # Strip timezone if present
        df["time"] = pd.to_datetime(df["time"]).dt.tz_localize(None)
        
        # Convert to just date (no time)
        df["date"] = df["time"].dt.date
        
        # Drop original time column
        df.drop(columns=["time"], inplace=True)
        
        # Reorder so 'date' is first column
        cols = ["date"] + [col for col in df.columns if col != "date"]
        df = df[cols]
        
        # Save with 'date' as a regular column (not index)
        df.to_csv(filepath, index=False)
        print(f"✔️ Cleaned and updated: {filename}")


Cleaning: AUD_USD.csv
✔️ Cleaned and updated: AUD_USD.csv
Cleaning: AUD_JPY.csv
✔️ Cleaned and updated: AUD_JPY.csv
Cleaning: EUR_AUD.csv
✔️ Cleaned and updated: EUR_AUD.csv
Cleaning: EUR_CAD.csv
✔️ Cleaned and updated: EUR_CAD.csv
Cleaning: USD_JPY.csv
✔️ Cleaned and updated: USD_JPY.csv
Cleaning: USD_CHF.csv
✔️ Cleaned and updated: USD_CHF.csv
Cleaning: GBP_CAD.csv
✔️ Cleaned and updated: GBP_CAD.csv
Cleaning: NZD_JPY.csv
✔️ Cleaned and updated: NZD_JPY.csv
Cleaning: NZD_USD.csv
✔️ Cleaned and updated: NZD_USD.csv
Cleaning: CHF_JPY.csv
✔️ Cleaned and updated: CHF_JPY.csv
Cleaning: GBP_USD.csv
✔️ Cleaned and updated: GBP_USD.csv
Cleaning: GBP_JPY.csv
✔️ Cleaned and updated: GBP_JPY.csv
Cleaning: EUR_CHF.csv
✔️ Cleaned and updated: EUR_CHF.csv
Cleaning: CAD_JPY.csv
✔️ Cleaned and updated: CAD_JPY.csv
Cleaning: AUD_NZD.csv
✔️ Cleaned and updated: AUD_NZD.csv
Cleaning: EUR_JPY.csv
✔️ Cleaned and updated: EUR_JPY.csv
Cleaning: EUR_GBP.csv
✔️ Cleaned and updated: EUR_GBP.csv
Cleaning: USD_